In [4]:
import json
import h3
import pandas as pd
import numpy as np
from typing import List, Dict, Any
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    두 좌표 사이의 거리를 미터 단위로 계산합니다.
    """
    R = 6371000  # 지구 반경 (미터)
    
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = (math.sin(delta_phi / 2) ** 2 +
         math.cos(phi1) * math.cos(phi2) *
         math.sin(delta_lambda / 2) ** 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c

def process_images_with_h3(images_data: List[Dict[Any, Any]], max_distance: float = 5.0) -> List[Dict[Any, Any]]:
    """
    H3 그리드 시스템을 사용하여 이미지 중복을 제거합니다.
    
    Args:
        images_data: 좌표가 포함된 이미지 데이터 리스트
        max_distance: 중복으로 간주할 최대 거리 (미터)
    
    Returns:
        중복이 제거된 이미지 리스트
    """
    # 5m 거리에 적합한 해상도 선택 (해상도 14)
    resolution = 14
    
    # 결과를 저장할 리스트
    unique_images = []
    
    # 이미 처리한 H3 인덱스를 추적
    processed_h3_indices = set()
    
    # 각 H3 인덱스에 대한 대표 이미지 저장
    h3_representative_images = {}
    
    for image in images_data:
        # JSON에서 좌표 추출 (필요에 따라 구조 조정)
        try:
            latitude = image.get('latitude') or image.get('location', {}).get('latitude')
            longitude = image.get('longitude') or image.get('location', {}).get('longitude')
            
            if latitude is None or longitude is None:
                # 좌표가 없으면 중복 체크 없이 추가
                unique_images.append(image)
                continue
                
            # 위도/경도를 H3 인덱스로 변환 (geo_to_h3 → latlng_to_cell)
            h3_index = h3.latlng_to_cell(latitude, longitude, resolution)
            
            # 이웃하는 H3 셀도 확인 (5m 거리 확인을 위해) (k_ring → grid_disk)
            neighbor_indices = h3.grid_disk(h3_index, 1)  # k=1: 자신과 직접 이웃한 셀들
            
            duplicate_found = False
            
            # 이웃 셀들을 검사하여 5m 이내의 이미지가 있는지 확인
            for neighbor_index in neighbor_indices:
                if neighbor_index in h3_representative_images:
                    rep_image = h3_representative_images[neighbor_index]
                    rep_lat = rep_image['latitude']
                    rep_lon = rep_image['longitude']
                    
                    # 거리 계산
                    distance = haversine_distance(latitude, longitude, rep_lat, rep_lon)
                    
                    if distance <= max_distance:
                        duplicate_found = True
                        break
            
            if not duplicate_found:
                # 중복이 없으면 이 이미지를 H3 셀의 대표로 저장
                h3_representative_images[h3_index] = image
                
                # 원본 이미지 데이터에 H3 인덱스 추가 (선택사항)
                image['h3_index'] = h3_index
                unique_images.append(image)
                
        except Exception as e:
            print(f"이미지 처리 중 오류 발생: {e}")
            # 오류가 발생한 경우에도 이미지 추가 (선택적)
            unique_images.append(image)
    
    print(f"총 {len(images_data)}개 이미지 중 {len(unique_images)}개 유일한 위치 발견")
    return unique_images

# 예제 사용법
if __name__ == "__main__":
    # 테스트 데이터 - 실제 데이터에 맞게 조정 필요
    sample_data = [
        {"id": "img1", "latitude": 37.5665, "longitude": 126.9780, "filename": "image/test_image_01.jpg"},
        {"id": "img2", "latitude": 37.56651, "longitude": 126.97801, "filename": "image/test_image_02.jpg"},  # 약 1m 차이
        {"id": "img3", "latitude": 37.5666, "longitude": 126.9781, "filename": "image/test_image_03.jpg"},  # 약 10m 차이
        {"id": "img4", "latitude": 35.6762, "longitude": 139.6503, "filename": "image/test_image_04.jpg"},
    ]
    
    # 5m 거리 내의 이미지를 중복으로 처리
    unique_images = process_images_with_h3(sample_data, max_distance=5.0)
    
    # 결과 출력
    for img in unique_images:
        print(f"ID: {img['id']}, H3: {img.get('h3_index')}, 위치: {img['latitude']}, {img['longitude']}")

총 4개 이미지 중 3개 유일한 위치 발견
ID: img1, H3: 8e30e1d8c0b1d2f, 위치: 37.5665, 126.978
ID: img3, H3: 8e30e1d8c0b1827, 위치: 37.5666, 126.9781
ID: img4, H3: 8e2f5a363ba005f, 위치: 35.6762, 139.6503
